In [1]:
import yfinance as yf
from datetime import date
from openpyxl import Workbook
from openpyxl.chart import (
    BarChart,
    StockChart,
    Reference,
    Series,
)
from openpyxl.chart.axis import DateAxis, ChartLines
from openpyxl.chart.updown_bars import UpDownBars


In [2]:
wb = Workbook()
ws = wb.active

In [3]:
data = yf.Ticker('2330.TW').history(period="14D")
data = data.drop(['Dividends', 'Stock Splits'], axis=1)
data

,Open,High,Low,Close,Volume
Date,,,,,
2022-09-26,443.0,454.0,443.0,446.5,32342267
2022-09-27,449.0,451.5,446.0,448.0,24961404
2022-09-28,447.0,449.0,438.0,438.0,34402652
2022-09-29,443.0,443.5,432.0,435.0,41094780
2022-09-30,424.5,427.5,422.0,422.0,48266800
2022-10-03,418.5,424.5,416.5,417.0,36044395
2022-10-04,429.5,432.0,427.0,429.0,40375000
2022-10-05,444.5,451.0,442.0,445.0,46496536
2022-10-06,450.5,451.0,447.0,451.0,30331834


In [4]:
rows = data.reset_index().values.tolist()
rows.insert(0, ['日期', '開盤價', '最高價', '最低價', '收盤價', '成交量'])
rows

[['日期', '開盤價', '最高價', '最低價', '收盤價', '成交量'],
 [Timestamp('2022-09-26 00:00:00'), 443.0, 454.0, 443.0, 446.5, 32342267],
 [Timestamp('2022-09-27 00:00:00'), 449.0, 451.5, 446.0, 448.0, 24961404],
 [Timestamp('2022-09-28 00:00:00'), 447.0, 449.0, 438.0, 438.0, 34402652],
 [Timestamp('2022-09-29 00:00:00'), 443.0, 443.5, 432.0, 435.0, 41094780],
 [Timestamp('2022-09-30 00:00:00'), 424.5, 427.5, 422.0, 422.0, 48266800],
 [Timestamp('2022-10-03 00:00:00'), 418.5, 424.5, 416.5, 417.0, 36044395],
 [Timestamp('2022-10-04 00:00:00'), 429.5, 432.0, 427.0, 429.0, 40375000],
 [Timestamp('2022-10-05 00:00:00'), 444.5, 451.0, 442.0, 445.0, 46496536],
 [Timestamp('2022-10-06 00:00:00'), 450.5, 451.0, 447.0, 451.0, 30331834],
 [Timestamp('2022-10-07 00:00:00'), 448.5, 448.5, 437.0, 438.0, 31941781],
 [Timestamp('2022-10-11 00:00:00'), 408.0, 412.5, 401.0, 401.5, 95793819],
 [Timestamp('2022-10-12 00:00:00'), 397.0, 400.0, 395.5, 397.5, 53570085],
 [Timestamp('2022-10-13 00:00:00'), 400.5, 403.0, 395.0,

In [5]:
for row in rows:
    ws.append(row)

In [6]:
# High-low-close
c1 = StockChart()
labels = Reference(ws, min_col=1, min_row=2, max_row=len(rows))
data = Reference(ws, min_col=3, max_col=5, min_row=1, max_row=len(rows))
c1.add_data(data, titles_from_data=True)
c1.set_categories(labels)
for s in c1.series:
    s.graphicalProperties.line.noFill = True
    
    # marker for close
    s.marker.symbol = "dot"
    s.marker.size = 5
c1.title = "最高價-最低價-收盤價"
c1.hiLowLines = ChartLines()

# Excel is broken and needs a cache of values in order to display hiLoLines :-/
from openpyxl.chart.data_source import NumData, NumVal
pts = [NumVal(idx=i) for i in range(len(data) - 1)]
cache = NumData(pt=pts)
c1.series[-1].val.numRef.numCache = cache

ws.add_chart(c1, "G10")

In [7]:
# Open-high-low-close
c2 = StockChart()
data = Reference(ws, min_col=2, max_col=5, min_row=1, max_row=len(rows))
c2.add_data(data, titles_from_data=True)
c2.set_categories(labels)
for s in c2.series:
    s.graphicalProperties.line.noFill = True
c2.hiLowLines = ChartLines()
c2.upDownBars = UpDownBars()
c2.title = "開盤價-最高價-最低價-收盤價"

# add dummy cache
c2.series[-1].val.numRef.numCache = cache

ws.add_chart(c2, "Q10")

In [8]:
# Create bar chart for volume

bar = BarChart()
data =  Reference(ws, min_col=6, min_row=1, max_row=len(rows))
bar.add_data(data, titles_from_data=True)
bar.set_categories(labels)

In [9]:
from copy import deepcopy

# Volume-high-low-close
b1 = deepcopy(bar)
c3 = deepcopy(c1)
c3.y_axis.majorGridlines = None
c3.y_axis.title = "股價"
b1.y_axis.axId = 20
b1.z_axis = c3.y_axis
b1.y_axis.crosses = "max"
b1 += c3

c3.title = "最高價 最低價 收盤價 成交量"

ws.add_chart(b1, "G27")

In [10]:
## Volume-open-high-low-close
b2 = deepcopy(bar)
c4 = deepcopy(c2)
c4.y_axis.majorGridlines = None
c4.y_axis.title = "股價"
b2.y_axis.axId = 20
b2.z_axis = c4.y_axis
b2.y_axis.crosses = "max"
b2 += c4

ws.add_chart(b2, "Q27")

wb.save("stock.xlsx")